In [ ]:
# -------------------------------------------------------------------------------------- #
# ---------- DISCRETE DIPOLE METHOD FOR UV-VIS CALCULATIONS FROM DIPOLE DATA ----------- #
# -------------------------------------------------------------------------------------- #

import os
import json
import pathlib
import numpy as np
from dda import DDA

In [ ]:
# GPU Device Config
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="2"

current_folder_path = pathlib.Path().absolute()
# simulate the octahedra with longer axis as 80 nm from the superellipsoid
# a smaller geometry (longer axis length of ~ 75 nm) is actually simulated due to the geometric error 
config = {'gpu_device': '/GPU:0',
          'dipole_length': 2.88,
          'min_wavelength': 0.4,
          'max_wavelength': 0.95,
          'num_wavelengths': 56,
          'ref_medium': 1.333,
          'rotation_steps': 10,
          'folder_path': None,
          'calculate_electricField': False,
          'ref_data': str(current_folder_path) + '/Au_ref_index.csv',
          'dipole_data': str(current_folder_path) + '/dipoleList.csv'
        }

   
config['folder_path'] = str(current_folder_path)
np_dda = DDA(config)
np_dda.run_DDA()
np_dda.plot_spectra()
np.savetxt("Results.csv",np.array(np_dda.C_cross_total)/np.pi/np_dda.c_rad**2,delimiter=",")

In [ ]:
results = np.loadtxt('Results.csv',delimiter=',')

In [ ]:
from scipy.interpolate import interp1d

wavelength_temp = np.linspace(0.4,0.95,56)
UV_sample = interp1d(wavelength_temp, results, kind='cubic',fill_value='extrapolate')
wavelengths = np.linspace(0.4,0.95,1000)

series=UV_sample(wavelengths)

np.savetxt('Q_data.csv',series,delimiter=',')
np.savetxt('Q_data_wavelength.csv',wavelengths,delimiter=',')